<a href="https://github.com/ternaustralia/cmrset-examples/blob/main/gee-examples/python/Example_4_Exporting_the_Image_Stack_for_a_Region.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View on GitHub"/></a>
&nbsp;
<a href="https://colab.research.google.com/github/ternaustralia/cmrset-examples/blob/main/gee-examples/python/Example_4_Exporting_the_Image_Stack_for_a_Region.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
&nbsp;
<a href="https://mybinder.org/v2/gh/ternaustralia/cmrset-examples/HEAD?labpath=gee-examples%2Fpython%2FExample_4_Exporting_the_Image_Stack_for_a_Region.ipynb" target="_parent"><img src="https://mybinder.org/badge_logo.svg" alt="Open In Binder"/></a>


# Example 4 - Exporting the Image Stack for a Region
This example illustrates how to export the CMRSET Landsat V2.2 imagery via Earth Engine for a study area in the Pilbara. Both ETa and QA bands are exported. We will also illustrate changing the projection of the exported images to GDA94 (EPSG:4283). This example can potentially generate gigabytes of data, so we demonstrate two different methods for exporting the data to suit your own storage capabilities. These methods are:

1.   **Via Google Drive** - This method is the simplest, however you may not be prepared to pay for a monthly subscription to upgrade your Google Drive storage limits if your export exceeds the free storage tier (20 GB).

2.   **Via Google Cloud Storage** - This method can incur a once-off fee (rather than a subscription) for the egress costs associated with downloading the imagery from Google Cloud Storage. This method is recommended when method 1) is not appropriate for your circumstances.

To run this example it is assumed you have [registered](https://signup.earthengine.google.com/) to [Google Earth Engine](https://earthengine.google.com/). Please feel free to adapt this example to meet your own requirements.

---
**Execution Time:** ~3 minutes

**Storage Required:** 172 MB (on Google Drive or Google Cloud Storage)

## Install geemap
The [geemap](https://geemap.org/) Python package is built upon [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [folium](https://github.com/python-visualization/folium) packages, and enables users to analyze and visualize Earth Engine datasets interactively within a Jupyter-based environment.  The geemap API also mimics the official Google Earth Engine Javascript Map API when making calls such as `Map.addLayer()`, `Map.setCenter()`, and `Map.centerObject()`, etc. This simplifies the translation between both languages if you are transitioning from Javascript to Python.

Note: the package ipyleaflet is not compatible with Google Colab, hence you will notice we specifically import the [folium implementation](https://geemap.org/foliumap/) in this example for broarder compatibility.

In [ ]:
# Installs the geemap package
import subprocess

try:
    import geemap.foliumap as geemap
except ImportError:
    print('Installing geemap...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

Installing geemap...


In [ ]:
import ee
import geemap.foliumap as geemap

## Authenticate and initialize
Since we are using [geemap](https://geemap.org/) in this example, we don't need to call the `ee.Authenticate()` function to authenticate or `ee.Initialize()` to initialize the Earth Engine package like previous examples. Instead, these calls are built into the the initialization of the geemap package via the `geemap.Map()` function. Similar to the previous examples though, upon running the following cell you'll be asked to grant Earth Engine access to your Google account. **Follow the instructions printed to the cell.**

In [ ]:
Map = geemap.Map(add_google_map = False)

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=i7i6E1Mj9IwGUHf7RmIvdAmTh6FcQ9QL9PqQCGVrHZw&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWj9HavgNbVImL8GSagPGqwyEdJqJsCvnyEvfmgAOFROsCWVoRYkJ6w

Successfully saved authorization token.


## Define the image stack
Firstly, define the extents of your region of interest. We will do this by specifying the extents of our rectangle.


In [ ]:
# Define a region of interest.
region = ee.Geometry.Rectangle([117.0, -21.5, 117.5, -21.0])

In [ ]:
# Visualize the region of interest.
Map.addLayer(region, {'color': 'green'}, 'Study Site')
Map.centerObject(region)
Map

Now, let's access [CMRSET](https://developers.google.com/earth-engine/datasets/catalog/TERN_AET_CMRSET_LANDSAT_V2_2) from the [Earth Engine data catalog](https://developers.google.com/earth-engine/datasets), and filter down the [ImageCollection](https://developers.google.com/earth-engine/tutorials/tutorial_api_04) to the period of interest. We will just export 12 months of data for this example, however you can customise the period of interest for your own circumstances.

In [ ]:
# Define the period of interest.
start = '2019-01-01'
end = '2020-01-01'

# Access evapotranspiration from the CMRSET dataset.
cmrset_aet = ee.ImageCollection("TERN/AET/CMRSET_LANDSAT_V2_2").filterDate(start, end)

## Export the result
Have you heard of the band 1023MB? They're not bad, but they haven't had any gigs yet...  Running the following cells will likely produce gigs of data (depending on your region defined and time period), so please check you have sufficient storage. We will demonstrate 2 different locations you can export the data to: [Google Drive](https://www.google.com/drive/); and [Google Cloud Storage](https://cloud.google.com/storage).  In both of these approaches we will also specify the output [projection](https://developers.google.com/earth-engine/guides/projections) as [GDA94](https://epsg.io/4283) (ESPG:4283)

In [ ]:
# Define the export settings.
site_name = 'Pilbara' # Description for your site (used in task descriptions/output file names)
export_folder = 'CMRSET_Exports'

###Export to Google Drive
This function defines an [export task](https://developers.google.com/earth-engine/guides/exporting) from Earth Engine which will [export an image to Google Drive](https://developers.google.com/earth-engine/guides/exporting#to-drive) as a [Cloud-Optimised GeoTIFF](https://www.cogeo.org/) (COG).

In [ ]:
tasks = []

# A helper function to export an image band to Google Drive
def exportBandToDrive(image, band_name):

  task_description = "{0}_{1}_{2}".format(site_name, feature['properties']['system:index'][0:10], band_name)
  task = ee.batch.Export.image.toDrive(**{
    'image': image.select(band_name),
    'region': region,
    'scale': 30,
    'crs': 'EPSG:4283',
    'description': task_description,
    'folder': export_folder
  })
  tasks.append(task)

### Export to Google Cloud Storage (GCS)
This function defines an [export task](https://developers.google.com/earth-engine/guides/exporting) from Earth Engine which will [export an image to GCS](https://developers.google.com/earth-engine/guides/exporting#to-cloud-storage) as a [Cloud-Optimised GeoTIFF](https://www.cogeo.org/) (COG).

Use this function if your preferred export location is to [Google Cloud Storage](https://cloud.google.com/storage).  You may export the data to an existing storage bucket or you can [create a new storage bucket](https://cloud.google.com/storage/docs/creating-buckets). If you are creating a new storage bucket you may also be interested in the [bucket naming guidelines](https://cloud.google.com/storage/docs/naming-buckets).  Depending on your circumstances, other information you may be interested in include: [downloading data from a bucket](https://cloud.google.com/storage/docs/downloading-objects); and [deleting buckets](https://cloud.google.com/storage/docs/deleting-buckets).

**You will need to adjust the value of the variable `myBucket` below to reference your bucket name.**



In [ ]:
tasks = []
myBucket = 'your-bucket-name'

# A helper function to export an image band to Google Cloud Storage
def exportBandToCloudStorage(image, band_name):

  task_description = "{0}_{1}_{2}".format(site_name, feature['properties']['system:index'][0:10], band_name)
  task = ee.batch.Export.image.toCloudStorage(**{
    'image': image.select(band_name),
    'region': region,
    'scale': 30,
    'crs': 'EPSG:4283',
    'description': task_description,
    'bucket': myBucket,
    'fileNamePrefix': "{0}/{1}/{2}".format(export_folder, site_name, task_description)
  })
  tasks.append(task)

Now, let's launch the tasks... by default, running the following cell will create export tasks to Google Drive for both the *ETa* and *pixel_qa* bands, however you can toggle the comments (with #) below to customise your exported data/location.

In [ ]:
eta_client = cmrset_aet.getInfo()

# Create an export task for each band.
for feature in eta_client['features']:
  
  # Get the image to export.
  image = cmrset_aet.filterDate(feature['properties']['system:time_start']).first()

  exportBandToDrive(image, 'ETa') # Evapotranspiration (to Google Drive)
  exportBandToDrive(image, 'pixel_qa') # Pixel QA (to Google Drive)

  #exportBandToCloudStorage(image, 'ETa')  # Evapotranspiration (to Cloud Storage)
  #exportBandToCloudStorage(image, 'pixel_qa')  # Pixel QA (to Cloud Storage)

# Start the tasks.
for task in tasks:
  print("Launching task: {0}".format(task.config['description']))
  task.start()

Launching task: Pilbara_2019_01_01_ETa
Launching task: Pilbara_2019_01_01_pixel_qa
Launching task: Pilbara_2019_02_01_ETa
Launching task: Pilbara_2019_02_01_pixel_qa
Launching task: Pilbara_2019_03_01_ETa
Launching task: Pilbara_2019_03_01_pixel_qa
Launching task: Pilbara_2019_04_01_ETa
Launching task: Pilbara_2019_04_01_pixel_qa
Launching task: Pilbara_2019_05_01_ETa
Launching task: Pilbara_2019_05_01_pixel_qa
Launching task: Pilbara_2019_06_01_ETa
Launching task: Pilbara_2019_06_01_pixel_qa
Launching task: Pilbara_2019_07_01_ETa
Launching task: Pilbara_2019_07_01_pixel_qa
Launching task: Pilbara_2019_08_01_ETa
Launching task: Pilbara_2019_08_01_pixel_qa
Launching task: Pilbara_2019_09_01_ETa
Launching task: Pilbara_2019_09_01_pixel_qa
Launching task: Pilbara_2019_10_01_ETa
Launching task: Pilbara_2019_10_01_pixel_qa
Launching task: Pilbara_2019_11_01_ETa
Launching task: Pilbara_2019_11_01_pixel_qa
Launching task: Pilbara_2019_12_01_ETa
Launching task: Pilbara_2019_12_01_pixel_qa


Now that your tasks have started, you may monitor their progress under the **Tasks** tab in the [Earth Engine Code Editor](https://code.earthengine.google.com/) or via the [Earth Engine Task Manager](https://code.earthengine.google.com/tasks)